# BGG Da2 Score 

The objetive is the get all Da2 guild user games Score

### Import Libaries

In [3]:
import requests as rq
import xml.etree.ElementTree as ET
import time 
import json
import pandas as pd
import sys
import pymongo
import json
import xml2json
from pymongo import MongoClient



### Mongo Conections

In [4]:
mongodb = MongoClient('localhost', 27017)

### Get Da2 Guilds Users

In [5]:
uri = 'http://www.boardgamegeek.com/xmlapi2/guild?id=862&members=1'
da2_guild = rq.get(uri)
while (da2_guild.status_code != 200):
    time.sleep(10)
    da2_guild = rq.get(uri)

In [6]:
# Check bgg results 

if da2_guild.status_code != 200:
    sys.exit()


In [7]:
# Parse Bgg 
da2_guild_parse = ET.fromstring(da2_guild.text.encode('utf-8'))

In [8]:
# Calculate Guild members pages 

guild_members_num = int(da2_guild_parse.find('members').get('count'))
num_pages = int(round(guild_members_num/25) + 1)





In [13]:
# Get Da2 guild Members list 

da2_guild_members = list()

for page in range(1, num_pages+1):
    uri = 'http://www.boardgamegeek.com/xmlapi2/guild?id=862&members=1&page='+str(page)
    da2_guild = rq.get(uri)
    while (da2_guild.status_code != 200):
        time.sleep(10)
        da2_guild = rq.get(uri)        
    
    # Vamos cogiendo los nombres de los usuarios
    for i in da2_guild_parse.find('members'):
        da2_guild_members.append(i.get('name'))

    

    

### Get Da2 Users games rates

In [6]:
da2_user = {}


for i in da2_guild_members:
    uri = 'http://www.boardgamegeek.com/xmlapi2/collection?username='+str(i)+'&stats=1&rated=1'
    bgg_user = rq.get(uri)
    while (bgg_user.status_code != 200):
        time.sleep(10)
        bgg_user = rq.get(uri)
        
    user_col = ET.fromstring(bgg_user.text.encode('utf-8'))
    
    for juego in user_col.findall('item'):
                 
        rating = juego.find("stats").find("rating")
        ranks = rating.find("ranks")
        for r in ranks.findall("rank"):
            posicion = r.get("value")
            tipo = r.get("name")
            if (tipo == "boardgame"):
              if(juego.find("originalname") == None ):
                nombre_juego = juego.find("name").text
              else:
                nombre_juego = juego.find("originalname").text

            nota = rating.get("value")
        user_rate = {}
        user_rate['name'] = str(i)
        user_rate['gamename'] = nombre_juego
        user_rate['rate'] = str(nota)
        
        mongodb.bgg.da2.insert_one(user_rate)
        # Save in a file 
        #with open('C:\\temp\\'+ str(i)+'_'+str(nombre_juego)+'_'+'rates.json', 'w') as outfile:
        #    outfile.write(json.JSONEncoder().encode(user_rate))
        

    
    




NameError: name 'da2_guild_members' is not defined

In [ ]:

mongodb.bgg.da2.count()